This notebook uses linear programming to minimize non-CO2 greenhouse gas emissions.

In [1]:
##Import required libraries for data manipulation and analyses
import pandas as pd
import numpy as np
import gc
import os
from scipy.optimize import linprog
gc.collect()

0

From https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html
Linear programming solves problems of the following form:

c @ x
such that:

A_ub @ x <= b_ub
A_eq @ x == b_eq
lb <= x <= ub

In [ ]:
##Scenario 1: Increase imports##
##Read the downloaded trade_enft dataframe. The dataframe depicts nutrient intensity (e.g., iron (mg) /energy (kcal)) of trade baskets 
##for every trade partnership (Importer-Exporter) and food group (i.e.i FBS Code2). 
trade_enft=pd.read_csv("trade_enft.csv")

##Make sure that the dataframe is sorted in ascending order for consistency throughout.
trade_enft2=trade_enft.groupby(['Reporter','ISO3','FBS Code2'],as_index=False)\
    ['Energy','Protein2', 'Iron2', 'Zinc2','Vitamin A', 'Vitamin B12', 'Folate2'].sum().sort_values(by= ['Reporter','ISO3','FBS Code2'],ascending=True)

##Choose the country of interest.
trade_enftnutr=trade_enft2[(trade_enft2['Reporter']=='AUT')&(trade_enft2['Energy']>0)]

##Create a trade basket dataframe for the chosen country.
tbasket_AUT=trade_enftnutr[['ISO3','FBS Code2']].drop_duplicates()

##Get the respective columns as a list of nutrient densities.
nutra=trade_enftnutr.loc[:,'Vitamin A'].to_list()
nutrb=trade_enftnutr.loc[:,'Vitamin B12'].to_list()
nutrpr=trade_enftnutr.loc[:,'Protein2'].to_list()
nutrir=trade_enftnutr.loc[:,'Iron2'].to_list()
nutrzn=trade_enftnutr.loc[:,'Zinc2'].to_list()
nutrft=trade_enftnutr.loc[:,'Folate2'].to_list()

##Multiply each element by -1. We need to convert our raw data into its negative equivalent because linprog function accepts only upper bound as an inequlity constraint.
nutra2=[i*(-1) if i>0 else i for i in nutra]
nutrb2=[i*(-1) if i>0 else i for i in nutrb]
nutrpr2=[i*(-1) if i>0 else i for i in nutrpr]
nutrir2=[i*(-1) if i>0 else i for i in nutrir]
nutrzn2=[i*(-1) if i>0 else i for i in nutrzn]
nutrft2=[i*(-1) if i>0 else i for i in nutrft]

##Read the downloaded opti_trade dataframe. It holds the emissions intensity of import baskets from each importer and by food group.
opti_trade=pd.read_csv("opti_trade.csv")
##Choose the county of interest. 
alldfnutr=opti_trade2[(opti_trade2['Reporter']=='AUT')&(opti_trade2['EnergyI-BaU']!=0)]

##Sort by ascending order for consistency.
alldfnutr2=alldfnutr.groupby(['Reporter','ISO3','FBS Code2'],as_index=False)['EnergyI-BaU']\
            .mean().sort_values(by= ['Reporter','ISO3','FBS Code2'],ascending=True)

##Create a vector for objective function. 
##The selected column represents the emissions intensities based on default emissions factors and current productivity and loss and waste patterns. 
obj=alldfnutr2['EnergyI-BaU']

##Apply the same steps  as above for estimates with the lower bound of emissions factors.
alldf_low=opti_trade2[(opti_trade2['Reporter']=='AUT')&(opti_trade2['EnergyI-BaU_low']!=0)]
alldf_low2=alldf_low.groupby(['Reporter','ISO3','FBS Code2'],as_index=False)['EnergyI-BaU_low'].mean().sort_values(by=['Reporter','ISO3','FBS Code2'],ascending=True)
obj_low=alldf_low2['EnergyI-BaU_low']

##Apply the same steps  as above for estimates with the higher bound of emissions factors.
alldf_high=opti_trade2[(opti_trade2['Reporter']=='AUT')&(opti_trade2['EnergyI-BaU_high']!=0)]
alldf_high2=alldf_high.groupby(['Reporter','ISO3','FBS Code2'],as_index=False)['EnergyI-BaU_high'].mean().sort_values(by= ['Reporter','ISO3','FBS Code2'],ascending=True)
obj_high=alldf_high2['EnergyI-BaU_high']

##Read the downloaded dataframes,each of which depicting supply gaps in 2030 by country.
##Energy gap
energy_supply7=pd.read_csv("energy_supply7.csv")
##Protein gap
protein_supply7=pd.read_csv("protein_supply7.csv")
##Iron gap
iron_supply7=pd.read_csv("iron_supply7.csv")
##Zinc gap
zinc_supply7=pd.read_csv("zinc_supply7.csv")
##Vitamin A gap
vitA_supply7=pd.read_csv("vitA_supply7.csv")
##Vitamin B12 gap
vitB_supply7=pd.read_csv("vitB_supply7.csv")
##Folate gap
folate_supply7=pd.read_csv("folate_supply7.csv")

##Construct right-hand side vectors for each nutrient constraint
##The column BaU_needed2030 involves supply gaps for 2030 populations based on the UN medium variant.
rhs6=vitB_supply7[vitB_supply7['ISO3']=='AUT']
rhs5=vitA_supply7[vitA_supply7['ISO3']=='AUT']
rhs1=energy_supply7[energy_supply7['ISO3']=='AUT']
rhs2=protein_supply7[protein_supply7['ISO3']=='AUT']
rhs3=iron_supply7[iron_supply7['ISO3']=='AUT']
rhs4=zinc_supply7[zinc_supply7['ISO3']=='AUT']
rhs7=folate_supply7[folate_supply7['ISO3']=='AUT']

##Construct the right hand side matrix of the objective function. 
##In this model, the right hand side represents the supply gap for seven nutrients.
## These are same across all scenarios because nutrient supplies and gaps to be filled are assumed to be constant.
myrhs2030=rhs1['BaU_needed2030']
myrhs20302=np.concatenate((myrhs2030,rhs2['BaU_needed2030'],rhs3['BaU_needed2030'],rhs4['BaU_needed2030'],rhs5['BaU_needed2030'],rhs6['BaU_needed2030'],rhs7['BaU_needed2030']))
##Multiply each element by -1 because linprog function accepts only upper bound as an inequlity constraint.
rhs2030=[i*(-1) for i in myrhs20302]

##Return a new array of length trade_enftnutr, filled with the values of nutrient density per unit energy. 
##This is needed to construct left hand side of the optimization model.
tr_lhs=[np.full(len(trade_enftnutr),-1), nutrpr2, nutrir2,nutrzn2,nutra2,nutrb2, nutrft2]

##Apply linprog function using the corresponding variables. Use "highs-ipm" method which utilizes interior point.
##See https://www.maths.ed.ac.uk/hall/HiGHS/ for further explanation of HiGHS 
tr_resipm2030_AUT=linprog(c=obj,A_ub=tr_lhs,b_ub=rhs2030,method='highs-ipm')
tr_resipm2030_AUT_low=linprog(c=obj_low,A_ub=tr_lhs,b_ub=rhs2030,method='highs-ipm')
tr_resipm2030_AUT_high=linprog(c=obj_high,A_ub=tr_lhs,b_ub=rhs2030,method='highs-ipm')

##Create dataframe of optimal trade baskets by importer - default emissions factors
xxx2030=pd.DataFrame(data=(tr_resipm2030_AUT.x.tolist()))
xxx20302=pd.concat([xxx2030,tbasket_AUT.reset_index()],ignore_index=True,axis='columns')
xxx20302.columns=['Imports','Reporter','Partner','FBS Code2']
xxx20302['Reporter']='AUT'
tbasket_AUT20302=xxx20302.copy()

##Create dataframe of optimal trade baskets by importer - lower bound emissions factors 
xxx2030_low=pd.DataFrame(data=(tr_resipm2030_AUT_low.x.tolist()))
xxx2030_low2=pd.concat([xxx2030_low,tbasket_AUT.reset_index()],ignore_index=True,axis='columns')
xxx2030_low2.columns=['Imports','Reporter','Partner','FBS Code2']
xxx2030_low2['Reporter']='AUT'
tbasket_AUT2030_low2=xxx2030_low2.copy()

##Create dataframe of optimal trade baskets by importer - upper bound emissions factors
xxx2030_high=pd.DataFrame(data=(tr_resipm2030_AUT_high.x.tolist()))
xxx2030_high2=pd.concat([xxx2030_high,tbasket_AUT.reset_index()],ignore_index=True,axis='columns')
xxx2030_high2.columns=['Imports','Reporter','Partner','FBS Code2']
xxx2030_high2['Reporter']='AUT'
tbasket_AUT2030_high2=xxx2030_high2.copy()


##Scenario 2&3: Increase domestic production & current productivity##
##Read the downloaded enft dataframe. The dataframe depicts nutrient intensity (e.g., iron (mg) /energy (kcal)) of production baskets 
##for country and food group (i.e.i FBS Code2). 
##Follow the same steps as Scenario 1
##Make sure that the dataframe is sorted in ascending order for consistency throughout.
enft2=enft.groupby(['ISO3','FBS Code2'],as_index=False)['Energy','Protein2', 'Iron2', 'Zinc2','Vitamin A', 'Vitamin B12', 'Folate2'].sum().sort_values(by= ['ISO3','FBS Code2'],ascending=True)

##Choose the country of interest.
enftnutr=enft2[(enft2['ISO3']=='AUT')&(enft2['Energy']>0)]

##Get the respective columns as a list of nutrient densities.
nutra=enftnutr.loc[:,'Vitamin A'].to_list()
nutrb=enftnutr.loc[:,'Vitamin B12'].to_list()
nutrpr=enftnutr.loc[:,'Protein2'].to_list()
nutrir=enftnutr.loc[:,'Iron2'].to_list()
nutrzn=enftnutr.loc[:,'Zinc2'].to_list()
nutrft=enftnutr.loc[:,'Folate2'].to_list()

##Multiply each element by -1. We need to convert our raw data into its negative equivalent because linprog function accepts only upper bound as an inequlity constraint.
nutra2=[i*(-1) if i>0 else i for i in nutra]
nutrb2=[i*(-1) if i>0 else i for i in nutrb]
nutrpr2=[i*(-1) if i>0 else i for i in nutrpr]
nutrir2=[i*(-1) if i>0 else i for i in nutrir]
nutrzn2=[i*(-1) if i>0 else i for i in nutrzn]
nutrft2=[i*(-1) if i>0 else i for i in nutrft]

##Create a production basket dataframe for the chosen country.
pbasket_AUT=enftnutr['FBS Code2'].values

##Read the downloaded alldf6 dataframe. 
alldf6=pd.read_csv("alldf6.csv")
##Choose the country of interest. 
alldfnutr=alldf6[(alldf6['ISO3']=='AUT')(alldf6['EnergyI-BaU']!=0)]

#Sort by ascending order for consisteny.
alldfnutr2=alldfnutr.groupby(['ISO3','FBS Code2'],as_index=False)['EnergyI-BaU','EnergyI-HalfW'].mean().sort_values(by= ['ISO3','FBS Code2'],ascending=True)

##Create a vector for objective function. 
##The selected column represents the emissions intensities based on default emissions factors and current productivity and loss and waste. 
obj=alldfnutr2['EnergyI-BaU']

##The selected column represents the emissions intensities based on default emissions factors, current productivity and half loss and waste. 
obj_half=alldfnutr2['EnergyI-HalfW']

#Follow the same steps for estimates with lower bound emissions factors
alllowdf6=pd.read_csv("alllowdf6.csv")
alldf_low=alllowdf6[(alllowdf6['ISO3']=='AUT')(alllowdf6['EnergyI-BaU_low']!=0)]
alldf_low2=alldf_low.groupby(['ISO3','FBS Code2'],as_index=False)['EnergyI-BaU_low','EnergyI-HalfW_low'].mean().sort_values(by= ['ISO3','FBS Code2'],ascending=True)
obj_low=alldf_low2['EnergyI-BaU_low']
obj_halflow=alldf_low2['EnergyI-HalfW_low']

#Follow the same steps for estiamtes with higher bound emissions factors
allhighdf6=pd.read_csv("allhighdf6.csv")
alldf_high=allhighdf6[(allhighdf6['ISO3']=='AUT')&(allhighdf6['EnergyI-BaU_high']!=np.inf)&(allhighdf6['EnergyI-BaU_high']!=0)]
alldf_high2=alldf_high.groupby(['ISO3','FBS Code2'],as_index=False)['EnergyI-BaU_high','EnergyI-HalfW_high'].mean().sort_values(by= ['ISO3','FBS Code2'],ascending=True)
obj_high=alldf_high2['EnergyI-BaU_high']
obj_halfhigh=alldf_high2['EnergyI-HalfW_high']

##Construct left hand side of the constraints based on nutrient density of energy production.
lhs=[np.full(len(enftnutr),-1), nutrpr2,  nutrir2, nutrzn2,nutra2,nutrb2, nutrft2]

##Construct right hand side of the constraints based on nutrient gaps by country.
## These are same across all scenarios because nutrient supplies and gaps to be filled are assumed to be constant.
##myrhs2030=rhs1['BaU_needed2030']
##myrhs20302=np.concatenate((myrhs2030,rhs2['BaU_needed2030'],rhs3['BaU_needed2030'],rhs4['BaU_needed2030'],rhs5['BaU_needed2030'],rhs6['BaU_needed2030'],rhs7['BaU_needed2030']))
##rhs2030=[i*(-1) for i in myrhs20302]

##Apply linprog function using the corresponding variables. Use "highs-ipm" method which utilizes interior point.
##Scenario 2: Increase domestic production & current productivity & current food loss and waste##
resipm2030_AUT=linprog(c=obj,A_ub=lhs,b_ub=rhs2030,method='highs-ipm')
resipm2030_AUT_low=linprog(c=obj_low,A_ub=lhs,b_ub=rhs2030,method='highs-ipm')
resipm2030_AUT_high=linprog(c=obj_high,A_ub=lhs,b_ub=rhs2030,method='highs-ipm')

##Create dataframe of optimal production baskets by country - default emissions factors
ipm_pbasket_2030AUT=pd.DataFrame(data=(resipm2030_AUT.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T
##Create dataframe of optimal production baskets by country - lower bound emissions factor
ipm_low_pbasket_2030AUT=pd.DataFrame(data=(resipm2030_AUT_low.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T
##Create dataframe of optimal production baskets by country - higher bound emissions factor
ipm_high_pbasket_2030AUT=pd.DataFrame(data=(resipm2030_AUT_high.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T

##Scenario 3: Increase domestic production & current productivity & half food loss and waste##
resipm_half2030_AUT=linprog(c=obj_half,A_ub=lhs,b_ub=rhs2030,method='highs-ipm')
resipm_half2030_AUT_low=linprog(c=obj_halflow,A_ub=lhs,b_ub=rhs2030,method='highs-ipm')
resipm_half2030_AUT_high=linprog(c=obj_halfhigh,A_ub=lhs,b_ub=rhs2030,method='highs-ipm')

##Create dataframe of optimal production baskets by country - default emissions factor
ipm_pbasket_half2030_AUT=pd.DataFrame(data=(resipm_half2030_AUT.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T
##Create dataframe of optimal production baskets by country - lower bound emissions factor
ipm_low_pbasket_half2030_AUT=pd.DataFrame(data=(resipm_half2030_AUT_low.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T
##Create dataframe of optimal production baskets by country - higher bound emissions factor
ipm_high_pbasket_half2030_AUT=pd.DataFrame(data=(resipm_half2030_AUT_high.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T


##Scenario 4&5: Increase domestic production & improved productivity##
##Read the downloaded yg_alldf6 dataframe.
yg_alldf6=pd.read_csv("yg_alldf6.csv")
##Choose the country of interest. 
yg_alldf_def=yg_alldf6[(yg_alldf6['ISO3']=='AUT')&(yg_alldf6['EnergyI-BaU']!=0)]

#Sort by ascending order for consisteny.
yg_alldf_def2=yg_alldf_def.groupby(['ISO3','FBS Code2'],as_index=False)['EnergyI-BaU','EnergyI-HalfW'].mean().sort_values(by= ['ISO3','FBS Code2'],ascending=True)

##Create a vector for objective function. 
##The selected column represents the emissions intensities based on default emissions factors and current productivity and loss and waste. 
yg_obj=yg_alldf_def2['EnergyI-BaU']

##The selected column represents the emissions intensities based on default emissions factors, improved productivity and current loss and waste. 
yg_obj_half=yg_alldf_def2['EnergyI-HalfW']

#Follow the same steps for estimates with lower bound emissions factors
yg_alldf_low=pd.read_csv("yg_alllowdf6.csv")
yg_alldf_low=yg_alllowdf6[(yg_alllowdf6['ISO3']=='AUT')(yg_alllowdf6['EnergyI-BaU_low']!=0)]
yg_alldf_low2=yg_alldf_low.groupby(['ISO3','FBS Code2'],as_index=False)['EnergyI-BaU_low','EnergyI-HalfW_low'].mean().sort_values(by= ['ISO3','FBS Code2'],ascending=True)
yg_obj_low=yg_alldf_low2['EnergyI-BaU_low']
yg_obj_halflow=yg_alldf_low2['EnergyI-HalfW_low']

#Follow the same steps for estimates with higher bound emissions factors
yg_alldf_high=pd.read_csv("yg_allhighdf6.csv")
yg_alldf_high=yg_allhighdf6[(yg_allhighdf6['ISO3']=='AUT')&(yg_allhighdf6['EnergyI-BaU_high']!=0)]
yg_alldf_high2=yg_alldf_high.groupby(['ISO3','FBS Code2'],as_index=False)['EnergyI-BaU_high','EnergyI-HalfW_high'].mean().sort_values(by= ['ISO3','FBS Code2'],ascending=True)
yg_obj_high=yg_alldf_high2['EnergyI-BaU_high']
yg_obj_halfhigh=yg_alldf_high2['EnergyI-HalfW_high']

##Apply linprog function using the corresponding variables. Use "highs-ipm" method which utilizes interior point.
##Scenario 4: Increase domestic production & improved productivity & current food loss and waste##
yg_resipm2030_AUT=linprog(c=yg_obj,A_ub=lhs,b_ub=rhs2030,method="highs-ipm")
yg_resipm2030_AUT_low=linprog(c=yg_obj_low,A_ub=lhs,b_ub=rhs2030,method="highs-ipm")
yg_resipm2030_AUT_high=linprog(c=yg_obj_high,A_ub=lhs,b_ub=rhs2030,method="highs-ipm")

##Create dataframe of optimal production baskets by country - default emissions factor
ipm_pbasket_yg_2030AUT=pd.DataFrame(data=(yg_resipm2030_AUT.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T
##Create dataframe of optimal production baskets by country - lower bound emissions factor
ipm_low_pbasket_yg_2030AUT=pd.DataFrame(data=(yg_resipm2030_AUT_low.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T
##Create dataframe of optimal production baskets by country - higher bound emissions factor
ipm_high_pbasket_yg_2030AUT=pd.DataFrame(data=(yg_resipm2030_AUT_high.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T

##Apply linprog function using the corresponding variables. Use "highs-ipm" method which utilizes interior point.
##Scenario 5: Increase domestic production & improved productivity & half food loss and waste##
yg_resipm_half2030_AUT=linprog(c=yg_obj_half,A_ub=lhs,b_ub=rhs2030,method="highs-ipm")
yg_resipm_half2030_AUT_low=linprog(c=yg_obj_halflow,A_ub=lhs,b_ub=rhs2030,method="highs-ipm")
yg_resipm_half2030_AUT_high=linprog(c=yg_obj_halfhigh,A_ub=lhs,b_ub=rhs2030,method="highs-ipm")

##Create dataframe of optimal production baskets by country - default emissions factor
ipm_pbasket_halfw_yg_2030AUT=pd.DataFrame(data=(yg_resipm_half2030_AUT.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T
##Create dataframe of optimal production baskets by country - lower bound emissions factor
ipm_low_pbasket_halfw_yg_2030AUT=pd.DataFrame(data=(yg_resipm_half2030_AUT_low.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T
##Create dataframe of optimal production baskets by country - upper bound emissions factor
ipm_high_pbasket_halfw_yg_2030AUT=pd.DataFrame(data=(yg_resipm_half2030_AUT_high.x.tolist(),pbasket_AUT.tolist()),index=["Energy","FBS Code2"],columns=['AUT']*len(pbasket_AUT)).T